## 여기부터 

In [1]:
import pandas as pd
import numpy as np
import os

## 🐬 파일 수정 시 버전 수정 필요 
- Major, Minor, Patch
- Major : 동일한 이름을 가진 feature 의 특성이 바뀌었을 경우
    - ` 예를 들어, ` age column 이 numeric 이었는데 str 로 바뀐 경우 
- Minor : 신규 feature 추가 
- Patch : 코드 오류 및 기타 단순 수정인 경우
    - ` 예를 들어, ` 동작상 오류 또는 파일 경로 수정

In [2]:
VERSION = "1.0.0"

In [3]:
data_path = './data/train'
train_df = pd.read_csv(os.path.join(data_path, 'train_ratings.csv')) # 전체 학습 데이터

In [4]:
unique_item_list = train_df["item"].unique()

In [5]:
direc_df  =  pd.read_csv(os.path.join(data_path, 'directors.tsv'), sep='\t') # director
genre_df  =  pd.read_csv(os.path.join(data_path, 'genres.tsv'), sep='\t')    # genres
title_df  =  pd.read_csv(os.path.join(data_path, 'titles.tsv'), sep='\t')    # titles
writer_df =  pd.read_csv(os.path.join(data_path, 'writers.tsv'), sep='\t')  # writers
year_df   =  pd.read_csv(os.path.join(data_path, 'years.tsv'), sep='\t')      # years


In [6]:
base_item_df = pd.DataFrame(unique_item_list, columns=["item"])

## 여기까지 

## ✨ 유진 : VERSION 1.0.0
- Item 속성 취합
    - Item 의 모든 특성을 취합하여 하나의 dataframe 으로 생성한다.

In [7]:
def get_double_item_id_list( target_df : pd.DataFrame, target_col : str = "item"):
    return target_df.groupby("item").size().loc[lambda x: x>1].sort_values().reset_index()["item"]

In [8]:
# 감독은 최대 14 명 까지 가능
double_direc_item_list = get_double_item_id_list(direc_df)
double_genre_item_list = get_double_item_id_list(genre_df)
double_writer_item_list = get_double_item_id_list(writer_df)
double_title_item_list = get_double_item_id_list(title_df)
double_year_item_list = get_double_item_id_list(year_df)

In [9]:
def concat_double_item_id_list(merge_df:pd.DataFrame,double_item_list : list, target_column : str) : 
    temp_item_df = pd.DataFrame(unique_item_list, columns=["item"])
    temp_item_df = temp_item_df.merge(merge_df,how='left',on = "item")
    temp_item_df = temp_item_df.drop_duplicates()
    
    for item in double_item_list : 
        name = ""
        info_list = merge_df[merge_df["item"]==item][target_column].to_list()
        for director in info_list:
            name+= director + ","
        temp_item_df.loc[ temp_item_df["item"] == item ,target_column] = name[:-1]
    
    return temp_item_df

In [10]:
concat_direc_df = concat_double_item_id_list(direc_df, double_direc_item_list, "director")
concat_genre_df = concat_double_item_id_list(genre_df, double_genre_item_list, "genre")
concat_writer_df = concat_double_item_id_list(writer_df, double_writer_item_list, "writer")

In [11]:
base_item_df = base_item_df.merge(concat_direc_df,how="left",on="item")
base_item_df = base_item_df.merge(concat_genre_df,how="left",on="item")
base_item_df = base_item_df.merge(concat_writer_df,how="left",on="item")

In [12]:
base_item_df = base_item_df.drop_duplicates()

In [13]:
base_item_df

,item,director,genre,writer
0,4643,nm0000318,"Action,Adventure,Drama,Sci-Fi","nm0099541,nm0115310,nm0465199,nm0742797"
16,170,nm0812200,"Action,Adventure,Crime,Thriller",NaN
20,531,nm0002140,"Children,Drama","nm0003031,nm0122364"
24,616,nm0718627,"Animation,Children","nm0027011,nm0166307,nm0166551,nm0314788,nm0841..."
38,2140,"nm0000568,nm0001345","Adventure,Fantasy","nm0001345,nm0643973"
...,...,...,...,...
37627,7753,nm0413875,Drama,nm0725564
37628,93422,NaN,Comedy,nm0779433
37629,6519,nm0467646,Comedy,"nm0069547,nm0301416,nm0318429,nm0725006"
37633,8830,nm0514546,"Adventure,Drama,Horror,Sci-Fi,Thriller","nm0061829,nm0143596,nm0163177,nm0258390,nm0591..."


## ✨ 이름 : VERSION , 수정 내용 기입 
> 수정 후 해당 셀을 마지막에 추가해주세요. 

## csv 파일 생성 

In [14]:
base_item_df.to_csv(f"FE_v{VERSION}.csv")